In [1]:
import pandas as pd
import numpy as np
import scipy

In [2]:
ncs = pd.read_csv('outgood.csv', sep=';')

In [3]:
VID_LEN = ncs.shape[0]
BINS_LEN = 120
labels = np.zeros((VID_LEN, BINS_LEN))
labels.shape

(4283, 120)

In [4]:
datapoints = ncs[[f'datapoint_{i}' for i in range(100)]].values
print(datapoints.shape)
durations = ncs['duration'].values
print(durations.shape)
t_steps = durations / 100
print(t_steps.shape)

(4283, 100)
(4283,)
(4283,)


In [5]:
DP_LEN = datapoints.shape[1]
timepoints = np.arange(DP_LEN)[np.newaxis, :] * t_steps[:, np.newaxis]
timepoints.shape

(4283, 100)

In [6]:
SONG_LEN = 240
BIN_TIME = 2

for i in range(VID_LEN):
    cubic_spline = scipy.interpolate.CubicSpline(timepoints[i], datapoints[i])
    data_interp = cubic_spline(np.arange(SONG_LEN) + 0.5)
    labels[i] = data_interp.reshape((BINS_LEN, BIN_TIME)).mean(axis=-1)

labels[np.arange(BINS_LEN)[np.newaxis, :] >= durations[:, np.newaxis] // BIN_TIME] = 0
labels[labels < 0] = 0
labels *= 1 / labels.max(axis=-1)[:, np.newaxis]
labels.shape

(4283, 120)

In [10]:
import pickle
import blosc

pickled_data = pickle.dumps(labels)
compressed_pickle = blosc.compress(pickled_data)

with open(f'timestamp_labels/timestamp_labels.dat', 'wb') as f:
    f.write(compressed_pickle)